# Подготовка данных
Необходимо создать excel или использовать готовую таблицу с некоторыми тестовыми данными

In [31]:
import pandas as pd
import math
import time

df = pd.read_excel("tests6.xlsx")
df.head()

,Id,Direction,Section,TestCaseName,Automated,Preconditions,Steps,Postconditions,ExpectedResult,TestData,Comments,Iterations,Priority,State,CreatedDate,CreatedById,Tags
0,544031.0,AI_Отладка,Пул задач пользователя,Контролер. Исполнить задачу,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Highest,Ready,05/23/2024 14:48:49,Служебный пользователь для Автотестов,смоук хф
1,NaN,NaN,NaN,NaN,NaN,"Роль:\n\n\nОперационист ТП, Контролер",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,Выполнен вход в ЕФР под ролью Контролер,NaN,NaN,Открывается страница поиска клиента,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,Предварительно инициировать/найти операцию дос...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,"Кликнуть на раздел ""Задачи"" в верхней части эк...",NaN,На странице задач отображается список задач(по...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Начало обработки

Удаляем ненужные столбцы, заполняем нужные ячейки дублирующими значениями

In [32]:
start_timer = time.time()

def clean_dataset(dataset):
    output_dataset = dataset.drop(['Automated', 'Iterations', 'Priority', 'State', 'CreatedDate', 'CreatedById', 'Tags', 'Comments', 'Direction', 'TestData'], axis=1, inplace=False)
    # заполняем столбики значениями
    for index, row in output_dataset.iterrows():
        if math.isnan(output_dataset.loc[index, 'Id']):
            output_dataset.loc[index, 'Id'] = output_dataset.loc[index - 1, 'Id']

        if pd.isna(output_dataset.loc[index, 'Section']):
            output_dataset.loc[index, 'Section'] = output_dataset.loc[index - 1, 'Section']

        if pd.isna(output_dataset.loc[index, 'TestCaseName']):
            output_dataset.loc[index, 'TestCaseName'] = output_dataset.loc[index - 1, 'TestCaseName']

    return output_dataset

output = clean_dataset(df)
end_timer = time.time()

print(end_timer - start_timer)
output.to_excel("output.xlsx", index = False)
output.head()

0.01675701141357422


,Id,Section,TestCaseName,Preconditions,Steps,Postconditions,ExpectedResult
0,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,NaN,NaN,NaN
1,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,"Роль:\n\n\nОперационист ТП, Контролер",NaN,NaN,NaN
2,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,Выполнен вход в ЕФР под ролью Контролер,NaN,NaN,Открывается страница поиска клиента
3,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,Предварительно инициировать/найти операцию дос...,NaN,NaN,NaN
4,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Кликнуть на раздел ""Задачи"" в верхней части эк...",NaN,На странице задач отображается список задач(по...


# Разбиваем тест-кейсы на отдельные DataFrame

На выходе получим массив датафреймов(тест-кейсов)

In [33]:
start_timer = time.time()

def parse_tests_by_id(dataset):
    test_cases = []
    start_index = 0
    current_id = dataset.loc[0, 'Id']  # Текущий Id для сравнения

    for index, row in dataset.iterrows():
        if row['Id'] != current_id:
            # Добавляем срез предыдущих строк как новый DataFrame
            test_cases.append(dataset.iloc[start_index:index].copy())
            start_index = index  # Обновляем начальный индекс для нового теста
            current_id = row['Id']  # Обновляем текущий Id

    # Добавляем последний тест
    if start_index < len(dataset):
        test_cases.append(dataset.iloc[start_index:].copy())

    return test_cases

end_timer = time.time()
print(end_timer - start_timer)
output_array = parse_tests_by_id(output)
output_array[0]

0.0


,Id,Section,TestCaseName,Preconditions,Steps,Postconditions,ExpectedResult
0,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,NaN,NaN,NaN
1,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,"Роль:\n\n\nОперационист ТП, Контролер",NaN,NaN,NaN
2,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,Выполнен вход в ЕФР под ролью Контролер,NaN,NaN,Открывается страница поиска клиента
3,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,Предварительно инициировать/найти операцию дос...,NaN,NaN,NaN
4,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Кликнуть на раздел ""Задачи"" в верхней части эк...",NaN,На странице задач отображается список задач(по...
5,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Нажать ""Взять"" для задачи из пула текущего пол...",NaN,Задаче назначается исполнитель – текущий польз...
6,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Кликнуть ""Мои задачи""",NaN,"Открывается пул задач, доступных на исполнение"
7,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Нажать ""Исполнить"" для задачи из пула текущего...",NaN,Открывается форма задачи в текущей вкладке


In [34]:
output_array[1]

,Id,Section,TestCaseName,Preconditions,Steps,Postconditions,ExpectedResult
8,544032.0,Пул задач пользователя,Контролер. Взять и исполнить задачу,NaN,NaN,NaN,NaN
9,544032.0,Пул задач пользователя,Контролер. Взять и исполнить задачу,Роль:\n\n\nОперационист ТП,NaN,NaN,NaN
10,544032.0,Пул задач пользователя,Контролер. Взять и исполнить задачу,Выполнен вход в ЕФР,NaN,NaN,Открывается страница поиска клиента
11,544032.0,Пул задач пользователя,Контролер. Взять и исполнить задачу,NaN,"Кликнуть ""Задачи"" в верхней части экрана",NaN,На форме отображается список задач доступных н...
12,544032.0,Пул задач пользователя,Контролер. Взять и исполнить задачу,NaN,"Нажать ""Взять и исполнить"" для задачи из пула ...",NaN,открывается форма задачи


In [35]:
output_array[2]

,Id,Section,TestCaseName,Preconditions,Steps,Postconditions,ExpectedResult
13,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,NaN,NaN,NaN,NaN
14,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,Роль:\n\n\nОперационист ТП\nМенеджер-операцион...,NaN,NaN,NaN
15,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,Выполнен вход в ЕФР,NaN,NaN,Открывается страница поиска клиента
16,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,"нажать ""Задачи""",NaN,NaN,NaN
17,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,В другой сессии браузера авторизоваться пользо...,NaN,NaN,Открывается страница поиска клиента
18,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,"Кликнуть ""Задачи"" в верхней части экрана",NaN,NaN,На форме отображается список задач доступных н...
19,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,"Нажать ""Взять"" для задачи из пула текущего пол...",NaN,NaN,NaN
20,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,NaN,Перейти на страницу входа в систему ЕФР https:...,NaN,Отображается страница входа в систему ЕФР
21,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,NaN,Ввести логин и пароль не заблокированного поль...,NaN,Вход осуществлен успешно. Отображается страниц...
22,544033.0,Пул задач пользователя,Контролер. 2 сессии Негатив,NaN,"Кликнуть ""Задачи"" в верхней части экрана",NaN,На форме отображается список задач доступных н...


# Удаление пустых ячеек

In [36]:
start_timer = time.time()

def remove_empty_cells(array):
    # обрабатываем Preconditions (удаляем первую ячейку)
    for dframe in array:
        if pd.isna(dframe.at[dframe.index[0], 'Preconditions']):
            for i in range(len(dframe['Preconditions'])):
                if i == len(dframe.index) - 1:
                    break
                else:
                    dframe.at[dframe.index[i], 'Preconditions'] = dframe.at[dframe.index[i + 1], 'Preconditions']
                    
        if pd.isna(dframe.at[dframe.index[0], 'Steps']):
            counter = 0
            for i in range(len(dframe['Steps'])):
                if pd.isna(dframe.at[dframe.index[i], 'Steps']):
                    counter += 1
                else:
                    break
            
            for i in range(len(dframe['Steps'])):
                if i + counter == len(dframe.index) - 1:
                    break
                else:
                    dframe.at[dframe.index[i], 'Steps'] = dframe.at[dframe.index[i + counter], 'Steps']

        if pd.isna(dframe.at[dframe.index[0], 'ExpectedResult']):
            counter = 0
            for i in range(len(dframe['ExpectedResult'])):
                if pd.isna(dframe.at[dframe.index[i], 'ExpectedResult']):
                    counter += 1
                else:
                    break

            for i in range(len(dframe['ExpectedResult'])):
                if i + counter == len(dframe.index) - 1:
                    break
                else:
                    dframe.at[dframe.index[i], 'ExpectedResult'] = dframe.at[dframe.index[i + counter], 'ExpectedResult']  
            
    return array

edited_output_array = remove_empty_cells(output_array)
end_timer = time.time()
print(end_timer - start_timer)
edited_output_array[0]

0.00400233268737793


,Id,Section,TestCaseName,Preconditions,Steps,Postconditions,ExpectedResult
0,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,"Роль:\n\n\nОперационист ТП, Контролер","Кликнуть на раздел ""Задачи"" в верхней части эк...",NaN,Открывается страница поиска клиента
1,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,Выполнен вход в ЕФР под ролью Контролер,"Нажать ""Взять"" для задачи из пула текущего пол...",NaN,NaN
2,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,Предварительно инициировать/найти операцию дос...,"Кликнуть ""Мои задачи""",NaN,На странице задач отображается список задач(по...
3,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,NaN,NaN,Задаче назначается исполнитель – текущий польз...
4,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Кликнуть на раздел ""Задачи"" в верхней части эк...",NaN,"Открывается пул задач, доступных на исполнение"
5,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Нажать ""Взять"" для задачи из пула текущего пол...",NaN,Задаче назначается исполнитель – текущий польз...
6,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Кликнуть ""Мои задачи""",NaN,"Открывается пул задач, доступных на исполнение"
7,544031.0,Пул задач пользователя,Контролер. Исполнить задачу,NaN,"Нажать ""Исполнить"" для задачи из пула текущего...",NaN,Открывается форма задачи в текущей вкладке


# spaCy

```conda install spacy```

```python -m spacy download ru_core_news_lg```

Тест маленькой модели

In [37]:
import spacy

start_timer = time.time()

spacy_nlp = spacy.load("ru_core_news_lg")

def vectorize_text(text, model):
    doc = model(text)
    return doc.vector

spacy_vectorized_output_array = []

for test_case in edited_output_array:
    spacy_vectorized_output_array.append(vectorize_text(str(test_case), spacy_nlp))
    
for i in range(len(spacy_vectorized_output_array)):
    pd.DataFrame(spacy_vectorized_output_array[i]).to_csv("spacy_output/spacy_vectorized_output_" + str(i) + ".csv", index = False)

end_timer = time.time()
print(end_timer - start_timer)
spacy_vectorized_output_array[0]

1.5180058479309082


array([-2.82260645e-02, -6.31809160e-02, -3.85782048e-02,  4.75937314e-02,
        9.37561505e-04,  7.45811388e-02,  1.10147856e-02,  3.98207158e-02,
        6.84837401e-02,  3.94725837e-02, -3.96377407e-02, -5.92696257e-02,
        4.46496382e-02,  1.47933261e-02, -2.75289807e-02,  2.83800829e-02,
       -3.49576138e-02, -7.23011866e-02,  2.01387680e-03,  2.17660200e-02,
        4.48117554e-02, -1.54204229e-02, -1.21551066e-05, -3.01781483e-03,
       -3.14293988e-02, -2.72353012e-02, -6.41582459e-02,  9.45593324e-03,
        1.90207604e-02, -2.88063623e-02,  2.78130956e-02,  8.97898246e-03,
        1.03822067e-01,  5.16171753e-02, -5.23341307e-03, -4.81839851e-02,
        3.60755585e-02,  6.65129721e-03,  1.32608425e-03, -5.62264211e-02,
        5.33076227e-02,  4.32200544e-02,  1.97379887e-02, -1.43129639e-02,
        3.09201516e-02, -6.98122233e-02, -3.13938819e-02, -3.63909304e-02,
       -3.67708206e-02, -1.57053508e-02, -5.33267036e-02, -3.90062109e-02,
       -3.59234735e-02, -

# sentence-transformers/paraphrase-multilingual-mpnet-base-v2

```conda install -c conda-forge sentence-transformers```

https://huggingface.co/sentence-transformers/paraphrase-multilingual-mpnet-base-v2

In [39]:
from sentence_transformers import SentenceTransformer

start_timer = time.time()

mpnet_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
mpnet_vectorized_output_array = []

for test_case in edited_output_array:
    mpnet_vectorized_output_array.append(mpnet_model.encode(str(test_case)))

for i in range(len(mpnet_vectorized_output_array)):
    pd.DataFrame(mpnet_vectorized_output_array[i]).to_csv("mpnet_output/mpnet_vectorized_output_" + str(i) + ".csv", index = False)

end_timer = time.time()
print(end_timer - start_timer)
mpnet_vectorized_output_array[0]

C:\Users\paul\.conda\envs\EmbeddingTest\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


3.856448173522949


array([-1.11211531e-01,  8.81950855e-02, -9.32812504e-03,  1.42898709e-02,
        6.49455190e-02,  9.18690264e-02,  1.01207852e-01,  1.04657426e-01,
        1.08247310e-01,  1.62565976e-01,  1.21688560e-01,  4.20409068e-02,
        4.14832234e-02,  4.42686342e-02, -4.67963405e-02,  5.05866930e-02,
        6.30102456e-02, -2.07532234e-02,  3.07556484e-02,  9.64561626e-02,
        2.44377740e-02, -1.43677294e-02,  6.75223116e-03, -3.75066623e-02,
       -9.39437822e-02,  7.96628278e-03,  6.36444837e-02,  1.96974277e-02,
        2.64382679e-02, -3.77826020e-02,  1.22141957e-01,  1.28821246e-02,
        3.36046517e-02, -1.61277708e-02, -7.51219166e-04, -3.29168374e-03,
        8.91334377e-04, -3.46914795e-03, -2.19326727e-02,  9.56537873e-02,
        7.56586865e-02,  4.43849806e-03,  5.90963587e-02,  2.09911540e-02,
       -9.62814912e-02, -1.46861281e-02, -1.43479526e-01,  2.14122087e-01,
        3.78074571e-02,  1.02434650e-01,  1.06375981e-02,  2.03113288e-01,
        1.83166668e-03,  

# google-bert/bert-base-multilingual-uncased 

https://huggingface.co/google-bert/bert-base-multilingual-uncased